In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:]  # width and length of petals
y = iris.target

tree = DecisionTreeClassifier(max_depth = 2)
tree.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [4]:
from sklearn.tree import export_graphviz
export_graphviz(
    tree, out_file="tree_class.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True, filled=True
)

We start at node 0, where we ask ourselfs if leaves length is smaller then 2,45.If yes, we go left, it's leaf node => we can end here and directly classify the object.If not, we go to the right node and check again if petal width is smaller than 1.75, if yes we classify it as Versicolor, otherwise we classify it as Verginica.

We can measure impunity(how much instances are incorrectly classified) with *gini*, which is computed as 1 - sum of squares(instance_classified/all instances).

Decision Tree model is called *white box model*, because it's intuitibally understandable, but something like neural networks and Random Forest are not and we can not intepret how they got their result, so we call them *black box plot*.

In [5]:
# we can make probabilities
print(tree.predict_proba([[5, 1.5]]))
# 0 % for one class, 90% for correct one class(because of impunity only 49/54) and 10% for incorrect one
print(tree.predict([[5, 1.5]]))

[[0.         0.90740741 0.09259259]]
[1]


Decision Trees are using CART(Classification And Regression Tree) algorithm, that in essence splits the data using a feature *k* and threshold *t*, the pair of which is determined according to the purest classification(the smaller the deviations, the better).Then it repeats the same algorithms repeteadly till it reaches the max_depth.

The algorithm is greedy(sees the optimum now without looking for concequences, which may lead to wrong optimum) and has high complexity, so it only works for small sets((n * m) best-fit features to check for log2(n) nodes to repeat).

There is another one measure for impunity, called *enthropy*, which is 0 when node leaf only contains one class, the higher we go, the bigger the impunity.Gini computes faster, so it's default, but enthropy is used to make trees more balanced

Decision Trees are very prone to overfitting, because they may little assumption about the data.So we are in heavy need to regurilize the non-parametric(because we didn't explicitly define the all parameters like in linear regresion) model.To do that we usually contrain minimums and maximums, aka max_depth, minimum samples leaf node must have and so on.Also, we can perfectly train the model and then prune the node with only leaves if it's imporvement in impuniry is not statistically significant.

## Regression
Algorithm also can work for regression tasks.In that case it takes the means of the area it is located at.In the end, it can be said, tha algorithm devides the data on rectangles and takes the mean of the values in that rectangle.It's also prone to overfitting, so we must be careful and apply regulization.

In [6]:
import numpy as np
m = 200
X = np.random.rand(m, 1)
y = 2 * (4 + X) ** 2
y = y + np.random.randn(m, 1) / 10

In [7]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth = 2)
tree_reg.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [8]:
import matplotlib.pyplot as plt

np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10


tree_reg1 = DecisionTreeRegressor(random_state=42, max_depth=2)
tree_reg2 = DecisionTreeRegressor(random_state=42, max_depth=3)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

def plot_regression_predictions(tree_reg, X, y, axes=[0, 1, -0.2, 1], ylabel="$y$"):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel("$x_1$", fontsize=18)
    if ylabel:
        plt.ylabel(ylabel, fontsize=18, rotation=0)
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

plt.figure(figsize=(11, 4))
plt.subplot(121)
plot_regression_predictions(tree_reg1, X, y)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
plt.text(0.21, 0.65, "Depth=0", fontsize=15)
plt.text(0.01, 0.2, "Depth=1", fontsize=13)
plt.text(0.65, 0.8, "Depth=1", fontsize=13)
plt.legend(loc="upper center", fontsize=18)
plt.title("max_depth=2", fontsize=14)

plt.subplot(122)
plot_regression_predictions(tree_reg2, X, y, ylabel=None)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
for split in (0.0458, 0.1298, 0.2873, 0.9040):
    plt.plot([split, split], [-0.2, 1], "k:", linewidth=1)
plt.text(0.3, 0.5, "Depth=2", fontsize=13)
plt.title("max_depth=3", fontsize=14)

plt.show()

<Figure size 1100x400 with 2 Axes>

But Decision trees is were sensetive algorithm.For example, it can overcomplex model just after rotating data for 50 degrees(because it fits orthogonals) and also change highly if some outliers(or data with high values) are removed.So, it doesn't generilize well, it will get different models for the same data because it's stohatic somewhat.But we can solve this problem taking average of all trees using RandomForest.

Decision Tree Regression is useful when there is some pattern in numeric data.For example, drug with 30ml works with 100% effectivity, 40 with 50% and with 0-20, 60-100 it works with close to 0% and 5% effectivity respectively.So, we can select the 4 means so that we can predict which effectivity we will get in each range.In the end we can even evaluate the model the same way as typical regression: we will use the sum of square errors.<br>
The only difference isthat out output will not be a discrete value, but a numeric.That's why we will take means: to have leaves that are akin to categories.We will classify data to 0%, 5%, 50% and 100% effectivnes.<br>
We can use it with categorical and numeric data, the idea is all the same as in classificator: we select as a node the variable, that will give us the lowest gini score.But we must find out how to work with numeric data, because we cann build many nodes with this type of data.So, our Tree will be build like this: do we have a dose with >= 29 ml?If yes, is a patient Female?If yes, than effectivity is 50%.Of course, we can have several dose nodes: is does higher than 13 and higher than 29?If second one is no, than its between 13 and 29 and we look ay effectivness according to this.<br>
We use the least square error to find the division of the data: we split data in the halves and see the errors according to the sum of distances from the mean of data points in each half.Note, that as final score we take residuals of **BOTH** halves.So, we evaluate the score for each threshold(that is moved by each datapoint) and select the minimum.This minimum will be exactly our division.Then we repeat the same algorithm for all remaining data points in other halves till our leaf node will contain less instances than a minimum for a decision node.Note, that we forget about left half with each time and evaluate the error for each iteration according to the data points left.<br>
We calculate with the means even for categorical data.And then we choose for the root(and other decision nodes) the one with the least SSR error.

## Ensemble methods and Random Forest

We make a predictor from the mean of the set of predictor.In other words, we train models on different subsets of training data and combine the predictors of each turn to make one big, more precise, predictor.Such method is called **Ensemle Learning**, because the group of predictors is called *ensemble*.<br>
In case of Decision Trees it's a Random Forest.

### Voting

One of such ensemble methods is *hard-voting classifier*.In general, it takes predictors of the same data, trained on different algorithms(that give around the same accuracy), and then makes a predictor based on voting: we throw the instance in each algorithm and predict the output.The one with majority of votes will be our final prediction.Suprisingly, this works well even for weak learnes(algorithm, that are only slightly better than random guessing) because of the law of large numbers, that with right amount of classifiers can make the prediction go up(if we have 1000, for example, we can go from 51% to 75%, we go infinetely precicer to the real value.<br>
The only issue is, we must make all our instances independent, or it will repeat the same mistakes.That's why exactly we use different algorithms.

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(noise=0.30, n_samples=500, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_reg = LogisticRegression()
rand_for = RandomForestClassifier(n_estimators=10)
svm_class = SVC(gamma='auto')

voting_class = VotingClassifier(
    estimators=[('log_reg', log_reg), ('random_for', rand_for), ('svm', svm_class)],
    voting="hard"
)
voting_class.fit(X_train, y_train)

C:\Users\1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


VotingClassifier(estimators=[('log_reg',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('random_for',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     crit...
                                        

In [11]:
from sklearn.metrics import accuracy_score
for classifier in (log_reg, rand_for, svm_class, voting_class):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(classifier.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.888
SVC 0.888
VotingClassifier 0.904


C:\Users\1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


We can estimate probabilities if we have `predict_proba()` method and can train voting for that.Such voting will be accepting the prediction with highest probability and such method is called *soft*(because the higher probability, the bigger the weight).In other words, we train classifiers not via their number of the smae categorical outputs, but with the highest probability.

### Bagging and Pasting

This algorithm takes random samples(they can be repeated) and random variables(we select if it's 1, 2, ... n variables training many models and choosing the one with the best perfomance) and select the one that perfoms the best) for root node and trains many DecisionTrees.Then, using the test set, we select as predictor the one with the highest votes for predictor.Such method is called *bagging*.This makes our data more flexible to new data in comprasion to Decision Trees.We can evaluate this Ensemble method, couting the error(or how much predictors were wrong) of the out-of-bag samples - samples, what were left our due to random selection.Usually, about 1/3 of the data is such type(not a bootstrapped randomly selected).So, the proportion of incrorrectly classified data is called "out-of-bag error"

We can impute the missing data in random forest algorithm with traditional most common/median/mean values or using the special method of refining the missing data with Random Forest.We select some point for this dataset(for example, yes or no, doesn't matter, or median for numeric values) and then build random forest using that assumption.Then we see which instances ended up in the same node.We count all instances for all Decision Trees to see which data points are highly correlated with missing data point.In the end, if it's categorical value, than we select based on highest score(take the same value from the instances, that was together with this data point most of the time), if it's numeric, we use weigts accroding to entrances/all instances.So, if 8 of all 10 instances were the value with the score 240, 1 time with data point with score 120 and so on.. we will sum it all up 240 * 0,8 + 120 * 0,1 +...

The pasting has the same idea, just this time we do not *replacemtnt*(we don't take the repeatance of the data points, we can't have exact same data point placed twice for our model).<br>
Also, the biggest advantage is that we can run the parallely on CPU.

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

# max_samples: how much samples used in bagging and n_jobs mean how much cores in CPU
# will be used, -1 means that all avaible.
bag_class = BaggingClassifier(DecisionTreeClassifier(),
max_samples=100, bootstrap=True, n_estimators=500, n_jobs=-1)
bag_class.fit(X_train, y_train)
y_predict = bag_class.predict(X_test)

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.912

In [14]:
# Let's evaluate using out-of-bag instances(oob)
bag_class = BaggingClassifier(DecisionTreeClassifier(),
max_samples=100, bootstrap=True, n_estimators=500, n_jobs=-1, oob_score=True)
bag_class.fit(X_train, y_train)
bag_class.oob_score_

0.928

In [15]:
from sklearn.ensemble import RandomForestClassifier

rand_f = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rand_f.fit(X_train, y_train)
y_predict_rf = rand_f.predict(X_test)

In [17]:
# equvivalent to above
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, n_jobs=-1, bootstrap=True
)

We can make our Random Trees even more random, if we refuse the best threshols selecting(which take many computations) and just take random thresholds.Such method is called Extremly Randomized Trees(or Extra-Trees).

Also, using Random trees is good to see the features' importance, because it's easy for us to determine looking at nodes how well our impurity score improved.So, we cand use RF `feature_importance_` for feature selection.

### Boosting
AdaBoost(Adaptative boosting) and Gradient Boosting

We call it AdaBoost, because we boost the weights to make adaptive model.<br>
AdaBoost has 3 main principles: 1) it's mostly made of stumps- node and two leaves(so it's a forest of stumps) 2) the errors of previous stump influence the next one 3) different stumps have different amount of say in the prediction <br>
It's mostly made of stumps because it's biggest advantage - to train the model on small datasets.<br>
We give each data point a weight, for the first iteration it's the same.First, we start with the variable, that classifies the data the best way.We compute how well it performs with each variable and compute the gini score.The variable with the tiniest gini score is our node.The amount of say the stump has in the prediction is determined by how well the classification was performed.We take the total error(how much datapoints were incorrectly guessed => we take the sum of their weights to count it), the the *Amount Of Say = 1/2 * log(1- Total error/ Total error)*.We can use istead of 1/2 any othe number between 0 and 1, because this scalar is the learning rate for our algorithm.<br>
Now we come to the second iteration.We must increase the weight of the incorrectly guessed values and decrese the values of correctly guessed.For that we multiply the previous weight for exponential in power of positive amount of say for incorrect and negative for correctly guessed values.The sum of the weight will not give us 1 because of our manipulations.Because of that we divide each weight by that sum(normilize them), so that they will give us approximately 1.<br>
Now we must give our next model more emphasis on incorrectly guessed datapoints.For that we can either use weighted gini index(the inccorect one weight more => more chances to select model, that prioterizes what we need) or we can make new data, using random numbers between 0 and 1 to select the same sized data sets: we select the value based on distribution(first data point weight is x, second is y => if <= x then we select the first data point, if x < value <= y then we select the second one and so on.Than on this dataset we can use normal Decision Trees algorithm(even the weights in new dataset are now the same).<br>
We repeate the same algorithm till we have overfit or our numbers of maximum iterations was hit.<br>
Now we can make predictions using each model's amount of say.We compare the sum of amounts of says of each decision and select the one with the best score.<br>
[video](https://www.youtube.com/watch?v=LsK-xG1cLYA)<br>
In Scikit-Learn we have also the function called SAMME(Stagewise Adaptive Modeling and Multicals Exponential loss function), which does the same as AdaBoost, but for more classed(AdaBoost only does for 2).If we can estimate probabilities, we can use SAMME.R(Real), which will rely on probability, that generally performs better.<br>
There is also variation for Regression, called `AdaBoostRegressor`

In [20]:
from sklearn.ensemble import AdaBoostClassifier
ada_boost = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), learning_rate=0.5,
    n_estimators=200, algorithm="SAMME.R"
)
ada_boost.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

Another boosting algorithm is Gradient Boost.<br>
Algorithm is almost the same as earlier one, but there are minor details.<br>
First, Gradient descent builds fixed-size trees, but they usually are larger than stupm.Second, it start with one leaf and builds next model according to residuals.Also, we have afixed learning rate(and have no amount of say).That said, we take average of the continues value and then count the residual for each instance, building a tree with the leaves, that are residuals.When value is descrete(aka it's a Gradient Classification), then we use the log(odds) of one class compared to the other.One leaf can contain many deviations from average, so we take their average.Than we can predict the value adding the difference multiplied by some learning rate(so that we won't have high variance) with average scores.When added we compute the deviation from computed num once again and based on that *pseudo residual* build the next model.So, we are making small steps in the right direction, which will stop either when we will perfectly fit the data or hit the maximum value of iterations.When making prediction, we go through all built trees, selected one pseudo residual to be added to compute the final score.<br>
One of the drawbacks of the Gradient Boost is that we must finish computing one tree first and can't run computations parallely.Also, I should note how Classification Gradient Boost is built.As I said, first we take the log odds and use it to conver value in probability.This probability will be our average.Than based on that we fit a line to the graph(that equals this probability) and compute ht pseudo residuals.We can't just add up probabilities or log odds terms, so we use formula *Sum of all residuals / sum of all previous possibilities * (1 - last probability)* to recieve new output, that will be added to the probability.Based on the thresholds, we will make our decisions(which is standardly 0.5, but based on trade-off can be altered).

In [21]:
from sklearn.tree import DecisionTreeRegressor

first_tree_reg = DecisionTreeRegressor(max_depth=2)
first_tree_reg.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [24]:
y1 = y - first_tree_reg.predict(X)
second_tree_reg = DecisionTreeRegressor(max_depth=2)
second_tree_reg.fit(X, y1)

y2 = y1 - second_tree_reg.predict(X)
third_tree_reg = DecisionTreeRegressor(max_depth=2)
third_tree_reg.fit(X, y2)

y_pred_final = sum(tree.predict(X) for tree in (first_tree_reg, second_tree_reg, third_tree_reg))

In [25]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbr.fit(X, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

The lower learning rate is, the more estimators we need, but models generalize better.This technique is called *shrinkage*.To find best amount of estimators we need to make we can use early stopping, like, for example, with `warm_start=True` parameter.We can also specify how much data we want to train on using `subsample`, which will reduce ariance, because we select training data more randomly.This approach is called *Stohatic Gradient Boosting*.

### Stacking

What makes the algorthm good is that it can be trained using varous methods as the base.That means, we train L estimators on training data and based on that output use meta-classifier on the second half of the data using outputs as input for predictions(y-axis).

### Extra-Trees

There is also another variation to the Random Forest, that give us more randomness and regulization, because rather than the best threshold, we have a random ones for each feature => less variance, althogh there is more bias.In other words, we still randomly select then number of features we consider for splits in node, but this time we don't select the best fit of all subsets of the size of this random numbers, but just one random.That gives us more time, altough sometimes it can lead to higher increse in bias, than in decrs in variance(when randomization is above optimum level).<br>
[here](https://stats.stackexchange.com/questions/175523/difference-between-random-forest-and-extremely-randomized-trees)